In [1]:
%cd ..

/home/hnguyen/nghia_branch/ourproject/equiRL


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import torchvision.models as model_pretrain

/home/hnguyen/miniconda3/envs/softgym/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
fi = np.load("data/RNN_imitation/video/demo_npy/data_1.npy", allow_pickle=True)

In [16]:
fi[0,3]

tensor([[[[216, 216, 216,  ..., 246, 246, 246],
          [216, 216, 216,  ..., 246, 246, 246],
          [216, 216, 216,  ..., 246, 246, 246],
          ...,
          [246, 246, 246,  ..., 216, 216, 216],
          [246, 246, 246,  ..., 216, 216, 216],
          [246, 246, 246,  ..., 216, 216, 216]],

         [[213, 213, 213,  ..., 243, 243, 243],
          [213, 213, 213,  ..., 243, 243, 243],
          [213, 213, 213,  ..., 243, 243, 243],
          ...,
          [243, 243, 243,  ..., 213, 213, 213],
          [243, 243, 243,  ..., 213, 213, 213],
          [243, 243, 243,  ..., 213, 213, 213]],

         [[213, 213, 213,  ..., 243, 243, 243],
          [213, 213, 213,  ..., 243, 243, 243],
          [213, 213, 213,  ..., 243, 243, 243],
          ...,
          [243, 243, 243,  ..., 213, 213, 213],
          [243, 243, 243,  ..., 213, 213, 213],
          [243, 243, 243,  ..., 213, 213, 213]]]], dtype=torch.uint8)

In [3]:
class TanhWrappedDistribution(torch.distributions.Distribution):
    """
    Class that wraps another valid torch distribution, such that sampled values from the base distribution are
    passed through a tanh layer. The corresponding (log) probabilities are also modified accordingly.
    Tanh Normal distribution - adapted from rlkit and CQL codebase
    (https://github.com/aviralkumar2907/CQL/blob/d67dbe9cf5d2b96e3b462b6146f249b3d6569796/d4rl/rlkit/torch/distributions.py#L6).
    """
    def __init__(self, base_dist, scale=1.0, epsilon=1e-6):
        """
        Args:
            base_dist (Distribution): Distribution to wrap with tanh output
            scale (float): Scale of output
            epsilon (float): Numerical stability epsilon when computing log-prob.
        """
        self.base_dist = base_dist
        self.scale = scale
        self.tanh_epsilon = epsilon
        super(TanhWrappedDistribution, self).__init__()

    def log_prob(self, value, pre_tanh_value=None):
        """
        Args:
            value (torch.Tensor): some tensor to compute log probabilities for
            pre_tanh_value: If specified, will not calculate atanh manually from @value. More numerically stable
        """
        value = value / self.scale
        if pre_tanh_value is None:
            one_plus_x = (1. + value).clamp(min=self.tanh_epsilon)
            one_minus_x = (1. - value).clamp(min=self.tanh_epsilon)
            pre_tanh_value = 0.5 * torch.log(one_plus_x / one_minus_x)
        lp = self.base_dist.log_prob(pre_tanh_value)
        tanh_lp = torch.log(1 - value * value + self.tanh_epsilon)
        # In case the base dist already sums up the log probs, make sure we do the same
        return lp - tanh_lp if len(lp.shape) == len(tanh_lp.shape) else lp - tanh_lp.sum(-1)
    def sample(self, sample_shape=torch.Size(), return_pretanh_value=False):
        """
        Gradients will and should *not* pass through this operation.
        See https://github.com/pytorch/pytorch/issues/4620 for discussion.
        """
        z = self.base_dist.sample(sample_shape=sample_shape).detach()

        if return_pretanh_value:
            return torch.tanh(z) * self.scale, z
        else:
            return torch.tanh(z) * self.scale

    def rsample(self, sample_shape=torch.Size(), return_pretanh_value=False):
        """
        Sampling in the reparameterization case - for differentiable samples.
        """
        z = self.base_dist.rsample(sample_shape=sample_shape)

        if return_pretanh_value:
            return torch.tanh(z) * self.scale, z
        else:
            return torch.tanh(z) * self.scale

In [4]:
class BC_RNN_GMM_actor(nn.Module):
    def __init__(self, obs_shape = (3,224,224), device = "cpu", node = 5):
        super(BC_RNN_GMM_actor, self).__init__()
        self.channel = obs_shape[0]
        self.device = device
        self.node = node
        self.encoder = model_pretrain.resnet50(pretrained = True)

        self.encoder.load_state_dict(torch.load("data/RNN_imitation/model/byol.pt"))

        for param in self.encoder.parameters():
            param.requires_grad = False

        self.per_step_mean = nn.Linear(200, node*8)
        self.per_step_std = nn.Linear(200, node*8)
        self.per_step_logit = nn.Linear(200, node)

        self.rnn = nn.LSTM(1100, 200 , 2, batch_first=True)

        self.relu = nn.ReLU()
    def forward(self, x, picker_state, init_state = None, train = True):
        # Process the input image with the CNN
        length = x.shape[1]
        x = x.view(x.size(0)*x.size(1),x.size(2),x.size(3),x.size(4))
        x = self.encoder(x)

        x = x.view(-1,length,x.size(1))

        x = torch.cat((x, picker_state), dim = 2)
        
        if init_state is None:
            h0 = torch.zeros(2,x.size(0),200).to(self.device)
            c0 = torch.zeros(2,x.size(0),200).to(self.device)
            init_state = (h0,c0)
        out, (hn,cn) = self.rnn(x, init_state)

        out = out.contiguous().view(-1,200)

        mean = self.per_step_mean(out)

        mean = mean.view(-1, length, self.node, 8)

        std = self.per_step_std(out)
        
        std = std.view(-1, length, self.node, 8)

        logits = self.per_step_logit(out)

        logits = logits.view(-1, length, self.node)


        if not train:
            std = torch.ones_like(mean) * 1e-4
        else:
            std = torch.nn.functional.softplus(std) + 1e-4

        component_distribution = torch.distributions.Normal(loc=mean, scale=std)
        component_distribution = torch.distributions.Independent(component_distribution, 1)
        mixture_distribution = torch.distributions.Categorical(logits=logits)

        dists = torch.distributions.MixtureSameFamily(
            mixture_distribution=mixture_distribution,
            component_distribution=component_distribution,
        )

        dists = TanhWrappedDistribution(base_dist=dists, scale=1.)

        return dists, hn, cn

In [5]:
model = BC_RNN_GMM_actor()

In [6]:
obs = torch.randn(1, 30, 3, 224, 224)
picker_state = torch.randn(1, 30, 100)

In [7]:
dists, hn, cn = model(obs, picker_state, train = True)